In [4]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
from IPython.display import display, Markdown

#import arcgis libraries
from arcgis.gis import *
gis = GIS()

In [12]:
working_directory = os.getcwd()
#data_folder =  os.path.join(working_directory, "data")
results_folder = os.path.join(working_directory, "results")

tract_filename = os.path.join(working_directory, r"tl_2018_49_tract\tl_2018_49_tract.shp")

tazcentroids_filename = working_directory + r"\TAZ\TAZCentroids.shp"

tazcentroids_zipcode_sj = working_directory + r"\Results\Shapefile\TAZCentroids_Tract.shp"


In [11]:
sdf_tract = pd.DataFrame.spatial.from_featureclass(tract_filename)

sdf_tract.head()

,FID,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,SHAPE
0,0,49,049,001001,49049001001,10.01,Census Tract 10.01,G5020,S,1693893.0,0.0,+40.2934782,-111.6812127,"{""rings"": [[[-111.695025, 40.297128], [-111.69..."
1,1,49,049,001002,49049001002,10.02,Census Tract 10.02,G5020,S,1508099.0,0.0,+40.2861635,-111.6792437,"{""rings"": [[[-111.691801, 40.289775], [-111.68..."
2,2,49,049,001103,49049001103,11.03,Census Tract 11.03,G5020,S,2084741.0,0.0,+40.2784944,-111.7079769,"{""rings"": [[[-111.724595, 40.282503], [-111.72..."
3,3,49,049,001201,49049001201,12.01,Census Tract 12.01,G5020,S,2463956.0,0.0,+40.2670294,-111.7036321,"{""rings"": [[[-111.718699, 40.27531], [-111.718..."
4,4,49,049,001202,49049001202,12.02,Census Tract 12.02,G5020,S,1697208.0,0.0,+40.2662170,-111.6886343,"{""rings"": [[[-111.695103, 40.273556], [-111.69..."


In [24]:
#verify that shape exists and sdf was created correctly and get spatial reference
print(sdf_tract.iloc[0].SHAPE.spatialReference)

{'wkid': 26912, 'latestWkid': 26912}


In [18]:
#change projection to 26912 to match TAZ Centroids
sdf_tract.spatial.project(26912)

True

In [25]:
#verify that shape exists and sdf was created correctly and get spatial reference
print(sdf_tract.iloc[0].SHAPE.spatialReference)

{'wkid': 26912, 'latestWkid': 26912}


In [13]:
sdf_tazcentroids = pd.DataFrame.spatial.from_featureclass(tazcentroids_filename)

sdf_tazcentroids.head()

,FID,TAZID,SORT,CO_TAZID,SUBAREAID,EXTERNAL,ACRES,DEVACRES,X,Y,...,DLRG_NAME,DISTMED,DMED_NAME,DISTSML,DSML_NAME,AIRSAGE,WF_AIRSAGE,SLC,NAAPM25,SHAPE
0,0,460,1,110460,1,0,322.29,322.29,409807.97148,4.551224e+06,...,North Davis,11,Clinton-W.Pt-Clearfield,98,Clinton-W.Pt-Clearfield,1106,13,2,NonAttnmnt|11|WFRC,"{""x"": 409807.9715, ""y"": 4551223.84, ""spatialRe..."
1,1,461,2,110461,1,0,316.91,316.91,409818.05584,4.552025e+06,...,North Davis,11,Clinton-W.Pt-Clearfield,98,Clinton-W.Pt-Clearfield,1106,13,2,NonAttnmnt|11|WFRC,"{""x"": 409818.0558000002, ""y"": 4552025.02050000..."
2,2,656,3,350656,1,0,828.83,701.91,407896.91458,4.518164e+06,...,North-west SL,17,NW Quad-International,155,NW Quad-International,3518,32,1,NonAttnmnt|35|WFRC,"{""x"": 407896.9145999998, ""y"": 4518164.2017, ""s..."
3,3,657,4,350657,1,0,531.35,531.28,409430.46088,4.518727e+06,...,North-west SL,17,NW Quad-International,155,NW Quad-International,3518,32,1,NonAttnmnt|35|WFRC,"{""x"": 409430.4609000003, ""y"": 4518726.88959999..."
4,4,2696,5,492696,1,0,575.28,574.81,435443.25244,4.435982e+06,...,Southwest,45,Payson-Salem,550,Payson-Salem,4907,43,2,NonAttnmnt|49|MAG,"{""x"": 435443.25239999965, ""y"": 4435981.8214, ""..."


In [15]:
#verify that shape exists and sdf was created correctly and get spatial reference
print(sdf_tazcentroids.iloc[0].SHAPE)

{'x': 409807.9715, 'y': 4551223.84, 'spatialReference': {'wkid': 26912, 'latestWkid': 26912}}


In [20]:
#spatial join tract centroid data to zip code data
sdf_tazcentroids_tract_join = sdf_tazcentroids.spatial.join(sdf_tract)

sdf_tazcentroids_tract_join.head()

,FID_left,TAZID,SORT,CO_TAZID,SUBAREAID,EXTERNAL,ACRES,DEVACRES,X,Y,...,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
0,0,460,1,110460,1,0,322.29,322.29,409807.97148,4.551224e+06,...,125401,49011125401,1254.01,Census Tract 1254.01,G5020,S,38851207.0,98152968.0,+41.1344047,-112.1791968
1,1,461,2,110461,1,0,316.91,316.91,409818.05584,4.552025e+06,...,125401,49011125401,1254.01,Census Tract 1254.01,G5020,S,38851207.0,98152968.0,+41.1344047,-112.1791968
2,446,425,447,110425,1,0,803.72,672.64,405293.60212,4.553193e+06,...,125401,49011125401,1254.01,Census Tract 1254.01,G5020,S,38851207.0,98152968.0,+41.1344047,-112.1791968
3,447,427,448,110427,1,0,1142.45,1140.09,407425.18568,4.550026e+06,...,125401,49011125401,1254.01,Census Tract 1254.01,G5020,S,38851207.0,98152968.0,+41.1344047,-112.1791968
4,475,457,476,110457,1,0,322.31,322.31,408210.77535,4.552846e+06,...,125401,49011125401,1254.01,Census Tract 1254.01,G5020,S,38851207.0,98152968.0,+41.1344047,-112.1791968


In [22]:
df_tazcentroids_tract = sdf_tazcentroids_tract_join[['TAZID','GEOID']].copy()
df_tazcentroids_tract

,TAZID,GEOID
0,460,49011125401
1,461,49011125401
2,425,49011125401
3,427,49011125401
4,457,49011125401
...,...,...
2876,898,49035113511
2877,1422,49035113527
2878,940,49035113532
2879,1439,49035112907


In [23]:
df_tazcentroids_tract.to_csv(os.path.join(results_folder, r"TAZ_CensusTract.csv"))